In [ ]:
import pandas
import os
from scripts.statement import Summary
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
pandas.set_option('display.max_colwidth', None)
pandas.options.display.float_format = "{:,.1f}".format
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists('img'):
    os.makedirs('img')

In [ ]:
S = Summary(years=["2025","2026"])

In [ ]:
S.update()

In [ ]:
df = S.projected.copy()
df.to_csv("data/{}/summary.csv".format(S.folders[-1]))
df

In [ ]:
for y in S.years:
    print(f"Year: {y}")
    tmp = df[[i for i in df.columns if y in i]]
    total_income = tmp[tmp.index.str.contains("Payroll|BBVA")].sum().sum()
    print(f"   Total income: ${total_income:,.1f}")
    total_savings = tmp.sum().sum()
    print(f"   Total savings: ${total_savings:,.1f}")
    print(f"   Average monthly income: ${total_income/12:,.1f}")
    print(f"   Average monthly savings: ${total_savings/12:,.1f}")
    print(f"   Average savings rate: {total_savings/total_income*100:,.1f}%")
    final = S.projected_balance[f"{y}-12"]
    print(f"   Final balance: ${final:,.1f}")


#### Subscriptions

In [ ]:
subscriptions = S.get_subscriptions()
subscriptions

## Report

In [ ]:
S.update()
fig,ax = plt.subplots(1,1,figsize=(10,5))
# Plot projected balance
S.projected_balance.plot(marker="o",color="black",linestyle="--",ax=ax)
print(f"Final balance: ${S.projected_balance.iloc[-1]:,.2f}")

# Plot actual balance
S.balance.plot(marker="o",color="black",ax=ax)
# Plot area plot with no lines
S.balances.T.plot.area(ax=ax,linewidth=0)

# Set the title and labels
_=ax.set_xticks(ticks=range(len(S.projected_balance)), labels=S.projected_balance.index, rotation=45)
# Plot with Y-axis formatted as currency
ax.yaxis.set_major_formatter('${x:,.0f}')
plt.ylabel("Balance")
plt.savefig("img/{}-projected.png".format(S.folders[-1]),format="PNG")

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(10,5))
categories = ["Groceries","Shopping","Food & Drink","SDGE","Travel"]
colors = ["#FD5F5F","#337CC6","#3BA13B","#C68B51","#C8B12D"]
S.projected.loc[categories].T.plot(marker="o",linestyle="--",ax=ax,color=colors,legend=False)
S.summary.loc[categories].T.plot(marker="o",ax=ax,color=colors)
_=ax.set_xticks(ticks=range(len(S.projected.columns)), labels=S.projected.columns, rotation=45)
ax.invert_yaxis()
ax.yaxis.set_major_formatter('{x:,.0f}')
plt.ylabel("Charge")
plt.savefig("./img/{}_individual.png".format(S.id),format="PNG")

## Changes

In [ ]:
df.sum()

In [ ]:
df.sum().plot(kind='bar', figsize=(6,5), title="Changes")
# Draw a line at current date
current_date = df.columns.to_list().index(S.folders[-1])
plt.axvline(x=current_date, color='r', linestyle='--', label='Current Date')
plt.legend()
plt.title("Projected Summary")
_=plt.xticks(rotation=45)

In [ ]:
S.update()
df = S.projected.copy()

In [ ]:
categories = "Payroll|Bills|Rent|Travel|Diezmo|Mama|Marky|Shopping|Food|Car|Internet"
df.loc[df.index.str.contains(categories)]
df.loc[df.index.str.contains(categories)].sum().plot(kind='bar', figsize=(6,5), title="Changes")

### Deltas

In [ ]:
S.deltas

In [ ]:
S.deltas.sum()

### Check a month

In [ ]:
s = S.get("2025-03")

In [ ]:
s.comparison.sort_values("delta")

In [ ]:
df = s.statement
df[df["type"].str.contains("Marky")]

### Average values all time

In [ ]:
S.summary.mean(axis=1)